In [16]:
from config.loadConfig import load_project_config
import os

from DB.Chroma import run_db
from sentence_transformers import SentenceTransformer
from RAG.RAG_Retrieve import custom_embed_function


In [14]:
config_yaml = load_project_config(os.path.join(os.getcwd(), "config/config.yaml"))

In [7]:
chroma_client = run_db()

In [15]:
embedding_model = SentenceTransformer(config_yaml['embedding_model'])

In [17]:
query_text = "Where is alabama?"
print(f"\n--- DEMO: Querying Small Chunks for: '{query_text}' ---")

# Encode the query text using the same model
query_embedding = custom_embed_function(embedding_model=embedding_model, texts=[query_text])


--- DEMO: Querying Small Chunks for: 'Where is alabama?' ---


In [25]:
from DB.Chroma import get_collection

small_collection = get_collection(
    client=chroma_client,
    collection_name=config_yaml['db']['collections']['collection_small']['name']
)

large_collection = get_collection(
    client=chroma_client,
    collection_name=config_yaml['db']['collections']['collection_large']['name']
)

# Search the small collection
retrieved_small_chunks = small_collection.query(
    query_embeddings=query_embedding,
    n_results=1,
    include=['metadatas','documents']
)

In [31]:
print(f"Retrieved small chunk:\n{retrieved_small_chunks['documents'][0][0]}")

Retrieved small chunk:
Alabama () is a state in the Southeastern region of the United States, bordered by Tennessee to the north; Georgia to the east; Florida and the Gulf of Mexico to the south; and Mississippi to the west. Alabama is the 30th largest by area and the 24th-most populous of the U.S. states.


In [33]:
parent_id = retrieved_small_chunks['metadatas'][0][0]['parent_id']
print(f"Retrieved Small Chunk Parent ID: {parent_id}")

full_context_result = large_collection.get(
        ids=[parent_id],
        include=['documents','metadatas']
)
print(full_context_result['documents'][0])
print(full_context_result['metadatas'])

Retrieved Small Chunk Parent ID: 53cab0f6-6494-4f48-9314-dcff65e6df9d
Alabama () is a state in the Southeastern region of the United States, bordered by Tennessee to the north; Georgia to the east; Florida and the Gulf of Mexico to the south; and Mississippi to the west. Alabama is the 30th largest by area and the 24th-most populous of the U.S. states.

Alabama is nicknamed the Yellowhammer State, after the state bird. Alabama is also known as the "Heart of Dixie" and the "Cotton State". The state tree is the longleaf pine, and the state flower is the camellia. Alabama's capital is Montgomery, and its largest city by population and area is Huntsville. Its oldest city is Mobile, founded by French colonists (Alabama Creoles) in 1702 as the capital of French Louisiana. Greater Birmingham is Alabama's largest metropolitan area and its economic center.
[{'doc_index': 3}]
